In [1]:
import pandas as pd
import math
pd.set_option('display.max_columns', None)
import os
import sys

In [2]:
# collect_attributes_db_failed.txt (collect_attributes_db)
# collect_attributes_failed.txt (collect_attributes)

In [3]:
# load datasets
columns = ['chunk_id', "failure_cause"]
df = pd.read_csv('../../data/logs/collect_attributes_db_failed.txt', delimiter = ":",  names=columns, header=None)
df2 = pd.read_csv('../../data/logs/collect_attributes_failed.txt', delimiter = ":",  names=columns, header=None)
selected = pd.read_csv('../../data/SELECTED_LABELLED_DATASET.csv')

chunks = pd.read_csv('../../data/LABELLED_DATASET.csv')

## Analyzing collect_attributes_db_failed.txt (collect_attributes_db)

In [4]:
df.head()

,chunk_id,failure_cause
0,1120635,REPO_NOT_AVAILABLE
1,1120636,REPO_NOT_AVAILABLE
2,1120637,REPO_NOT_AVAILABLE
3,1120638,REPO_NOT_AVAILABLE
4,1410716,REPO_NOT_AVAILABLE


In [5]:
# filter by selected projects only

print(f"Size of selected chunks: {len(selected)}")
selected.sample(n=3)
selected_chunks = list(selected['chunk_id'])
failed_selected = df[df['chunk_id'].isin(selected_chunks)]
print(f"Size of failed_selected: {len(failed_selected)}")
print(f"Size of failed: {len(df)}")
failed_selected.sample(n=3)

Size of selected chunks: 57214
Size of failed_selected: 3299
Size of failed: 6681


,chunk_id,failure_cause
1830,1380730,CANT_MERGE
5468,1150439,CANT_MERGE
905,1376577,CANT_MERGE


In [6]:
failed_selected['failure_cause'].value_counts()

CANT_MERGE                  3261
INCOSISTENT_MERGE_REPLAY      38
Name: failure_cause, dtype: int64

In [7]:
df['failure_cause'].value_counts()

REPO_NOT_AVAILABLE          3363
CANT_MERGE                  3261
INCOSISTENT_MERGE_REPLAY      52
INVALID_FILE                   5
Name: failure_cause, dtype: int64

In [8]:
failed_selected[failed_selected['failure_cause'] == 'CANT_MERGE'].sample(n=10)

,chunk_id,failure_cause
1572,1380408,CANT_MERGE
2858,931951,CANT_MERGE
5447,1150402,CANT_MERGE
907,1376579,CANT_MERGE
336,1375850,CANT_MERGE
1299,1380075,CANT_MERGE
1223,1379999,CANT_MERGE
5804,1150901,CANT_MERGE
970,1376643,CANT_MERGE
58,1375407,CANT_MERGE


In [9]:
chunks[chunks['chunk_id'] == 1149568]
# chunks.dtypes
# chunks.head()

,chunk_id,developerdecision,line_start,line_end,line_separator,kind_conflict,url,project,project_user,project_name,path,file_name,sha,leftsha,rightsha,basesha
100494,1149568,Manual,97,111,104,"Attribute, Method declaration, Method signatur...",https://github.com/revolsys/com.revolsys.open,revolsys/com.revolsys.open,revolsys,com.revolsys.open,revolsys/com.revolsys.open/com.revolsys.open.c...,CopyProcess.java,054624c44d26a26165ffbd1be05b1dbfed7b8373,ffe2430f2ecf5d478d07bf56f0d974ddd4035f7c,cce1ee64e3f2fa692d03dae9ad3fbacf374d279b,46f6e17dff46cdca056f246b1a8194902ca038de


## Analyzing collect_attributes_failed.txt (collect_attributes)

In [10]:
# load datasets
columns = ['chunk_id', "failure_cause"]
df2 = pd.read_csv('../../data/logs/collect_attributes_failed.txt', delimiter = ":",  names=columns, header=None)
selected = pd.read_csv('../../data/SELECTED_LABELLED_DATASET.csv')

chunks = pd.read_csv('../../data/LABELLED_DATASET.csv')




In [11]:
# LEMBRAR QUE ESSA EXTRAÇÃO SÓ CONSIDERA UM CHUNK DE CADA MERGE. ENTÃO A FALHA DE UM CHUNK VALE PARA TODOS OS CHUNKS
#DO MERGE DAQUELE CHUNK
new_chunks = set()
for index, row in df2.iterrows():
    chunk_id = row['chunk_id']
    failure_cause = row['failure_cause']
    merge_SHA = chunks[chunks['chunk_id'] == chunk_id].iloc[0]['sha']
    merge_chunks = list(chunks[chunks['sha'] == merge_SHA]['chunk_id'])
    new_chunks.add((chunk_id, failure_cause))
    for chunk in merge_chunks:
        new_chunks.add((chunk, failure_cause))
df2 = pd.DataFrame(list(new_chunks), columns=df2.columns)
df2.sample(n=3)
    

,chunk_id,failure_cause
1892,1172906,REPO_NOT_AVAILABLE
2453,947013,REPO_NOT_AVAILABLE
282,777653,REPO_NOT_AVAILABLE


In [12]:
len(df2)

3266

In [13]:
df2.head()

,chunk_id,failure_cause
0,1488498,REPO_NOT_AVAILABLE
1,1400690,REPO_NOT_AVAILABLE
2,1055407,REPO_NOT_AVAILABLE
3,1055145,REPO_NOT_AVAILABLE
4,1010048,REPO_NOT_AVAILABLE


In [14]:
# filter by selected projects only

print(f"Size of selected chunks: {len(selected)}")
selected.sample(n=3)
selected_chunks = list(selected['chunk_id'])
failed_selected = df2[df2['chunk_id'].isin(selected_chunks)]
print(f"Size of failed_selected: {len(failed_selected)}")
print(f"Size of failed: {len(df2)}")
# failed_selected.sample()

Size of selected chunks: 57214
Size of failed_selected: 0
Size of failed: 3266


In [15]:
df2['failure_cause'].value_counts()

REPO_NOT_AVAILABLE    3216
BAD_COMMIT              50
Name: failure_cause, dtype: int64